In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

In [2]:
# Fonction pour télécharger les prix d'une action
def get_stock_data(ticker, start, end):
    stock_data = yf.download(ticker, start=start, end=end)
    return stock_data


In [3]:
# Demander à l'utilisateur d'entrer les tickers à analyser
#tickers = input("Entrez les tickers des actions séparés par des espaces (ex: )MC.PA AIR.PA OR.PA SAN.PA BNP.PA: ").split()
tickers = [
    "AC.PA", "AI.PA", "AIR.PA", "MT.AS", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA",
    "CA.PA", "ACA.PA", "BN.PA", "DSY.PA", "EDEN.PA", "ENGI.PA", "EL.PA", "ERF.PA",
    "RMS.PA", "KER.PA", "OR.PA", "LR.PA", "MC.PA", "ML.PA", "ORA.PA", "RI.PA",
    "PUB.PA", "RNO.PA", "SAF.PA", "SGO.PA", "SAN.PA", "SU.PA", "GLE.PA", "STLAP.PA",
    "STMPA.PA", "TEP.PA", "HO.PA", "TTE.PA", "URW.PA", "VIE.PA", "DG.PA", "VIV.PA"
    ]

# Choix de la période de temps
#start_date = input("Entrez la date de début (YYYY-MM-DD, ex: 2010-01-01) : ").strip()
#end_date = input("Entrez la date de fin (YYYY-MM-DD, ex: 2024-01-01) : ").strip()
start_date = '2010-01-01'
end_date = '2025-01-01'

# Vérification des dates entrées
if start_date >= end_date:
    raise ValueError("La date de début doit être antérieure à la date de fin.")


In [4]:
# Télécharger les données historiques
stock_prices = get_stock_data(tickers, start_date, end_date)

# Réorganiser les colonnes pour Tableau
stock_prices = stock_prices.stack(future_stack=True).reset_index()
stock_prices.columns = ["Date", "Ticker"] + list(stock_prices.columns[2:])  # Dynamique pour éviter des erreurs



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  40 of 40 completed


In [ ]:
# Calcul du rendement journalier
stock_prices["Daily Return"] = stock_prices.groupby("Ticker")["Close"].pct_change(fill_method=None)

# Calcul du rendement cumulé
stock_prices["Cumulative Return"] = (1 + stock_prices["Daily Return"]).groupby(stock_prices["Ticker"]).cumprod()

# Calcul des moyennes mobiles à 50 et 200 jours
stock_prices["SMA_50"] = stock_prices.groupby("Ticker")["Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
stock_prices["SMA_200"] = stock_prices.groupby("Ticker")["Close"].rolling(window=200).mean().reset_index(level=0, drop=True)

# Définition du taux sans risque annuel (ex: 2%)
risk_free_rate_annual = 0.02  # 2%

# Conversion en taux sans risque journalier
risk_free_rate_daily = (1 + risk_free_rate_annual) ** (1/252) - 1  

# Calcul du Sharpe Ratio ajusté
stock_prices["Sharpe Ratio"] = stock_prices.groupby("Ticker")["Daily Return"].transform(
    lambda x: (x.mean() - risk_free_rate_daily) / (x.std() + 1e-8)
)

# Gérer les NaN
stock_prices.bfill(inplace=True)  # Backfill pour éviter les pertes de données
stock_prices.fillna(0, inplace=True)  # Dernière ligne de sécurité contre les NaN


In [7]:
# Dictionnaire pour stocker les données
sector_industry_dict = {}

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        sector = info.get("sector", "Unknown")   # Récupérer le secteur
        industry = info.get("industry", "Unknown")  # Récupérer l'industrie
        sector_industry_dict[ticker] = {"Sector": sector, "Industry": industry}
    except Exception as e:
        print(f"Erreur avec {ticker}: {e}")

# Transformer en DataFrame
sector_industry_df = pd.DataFrame.from_dict(sector_industry_dict, orient="index")
sector_industry_df.reset_index(inplace=True)
sector_industry_df.columns = ["Ticker", "Sector", "Industry"]

stock_prices = stock_prices.merge(sector_industry_df, on="Ticker", how="left")

In [8]:
# Exporter le CSV final
stock_prices.to_csv("stock_data_with_sectors.csv", index=False)

print("Fichier CSV créé : 'stock_data_with_sectors.csv' avec Secteur et Industrie")

Fichier CSV créé : 'stock_data_with_sectors.csv' avec Secteur et Industrie


In [9]:
stock_prices

,Date,Ticker,Close,High,Low,Open,Volume,Daily Return,Cumulative Return,SMA_50,SMA_200,Sharpe Ratio,Sector,Industry
0,2010-01-04,AC.PA,21.070032,21.070032,20.598680,20.631187,868104.0,-0.006171,0.993829,20.117249,20.912096,0.016927,Consumer Cyclical,Lodging
1,2010-01-04,ACA.PA,6.184280,6.196335,5.986576,6.003453,6105758.0,-0.006171,0.993829,20.117249,20.912096,0.025685,Financial Services,Banks - Regional
2,2010-01-04,AI.PA,30.840052,30.840052,30.340042,30.340042,1114568.0,-0.006171,0.993829,20.117249,20.912096,0.024293,Basic Materials,Specialty Chemicals
3,2010-01-04,AIR.PA,11.286739,11.335146,11.214130,11.294807,2496013.0,-0.006171,0.993829,20.117249,20.912096,0.028777,Industrials,Aerospace & Defense
4,2010-01-04,BN.PA,27.860062,27.930969,27.524864,27.540973,3685834.0,-0.006171,0.993829,20.117249,20.912096,0.016657,Consumer Defensive,Packaged Foods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153675,2024-12-31,TEP.PA,83.120003,83.900002,81.879997,81.879997,92904.0,0.017132,4.713559,90.085600,96.546660,0.026057,Industrials,Specialty Business Services
153676,2024-12-31,TTE.PA,52.579998,52.579998,51.791841,51.841100,1503880.0,0.014639,2.768798,54.965951,59.993398,0.019150,Energy,Oil & Gas Integrated
153677,2024-12-31,URW.PA,72.720001,73.139999,71.820000,71.820000,129597.0,0.005531,6.387704,74.853600,74.494865,0.025672,Real Estate,REIT - Retail
153678,2024-12-31,VIE.PA,27.110001,27.209999,26.770000,26.799999,764730.0,0.010060,2.381335,28.183400,28.846937,0.022005,Industrials,Waste Management
